In [1]:
import GEOparse
import pandas as pd
import numpy as np

In [2]:
import pylab as pl
import seaborn as sns
pl.rcParams['figure.figsize'] = (14, 10)
pl.rcParams['ytick.labelsize'] = 12
pl.rcParams['xtick.labelsize'] = 11
pl.rcParams['axes.labelsize'] = 23
pl.rcParams['legend.fontsize'] = 20
sns.set_style('ticks')
c1, c2, c3, c4 = sns.color_palette("Set1", 4)

In [3]:
gse = GEOparse.get_GEO("GSE84422")

02-Jun-2019 22:23:52 DEBUG utils - Directory ./ already exists. Skipping.
02-Jun-2019 22:23:52 INFO GEOparse - File already exist: using local version.
02-Jun-2019 22:23:52 INFO GEOparse - Parsing ./GSE84422_family.soft.gz: 
02-Jun-2019 22:23:52 DEBUG GEOparse - DATABASE: GeoMiame
02-Jun-2019 22:23:52 DEBUG GEOparse - SERIES: GSE84422
02-Jun-2019 22:23:52 DEBUG GEOparse - PLATFORM: GPL96
02-Jun-2019 22:23:53 DEBUG GEOparse - PLATFORM: GPL97
02-Jun-2019 22:23:53 DEBUG GEOparse - PLATFORM: GPL570
/home/ranap/anaconda3/envs/BioSandbox/lib/python3.6/site-packages/GEOparse/GEOparse.py:84: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return parse_GSE(filepath)
02-Jun-2019 22:23:55 DEBUG GEOparse - SAMPLE: GSM2233519
02-Jun-2019 22:23:56 DEBUG GEOparse - SAMPLE: GSM2233520
02-Jun-2019 22:23:56 DEBUG GEOparse - SAMPLE: GSM2233521
02-Jun-2019 22:23:56 DEBUG GEOparse - SAMPLE: GSM2233522
02-Jun-2019 22:23:56 DEBUG GEOparse - SAMPLE: GSM223

In [4]:
gse.metadata['title']

["Molecular Signatures Underlying Selective Regional Vulnerability to Alzheimer's Disease"]

In [5]:
gse.metadata['summary']

["Alzheimer's disease (AD) is the most common form of dementia, characterized by progressive cognitive impairment and neurodegeneration as a result of abnormal neuronal loss. To elucidate the molecular systems associated with AD, we characterized the gene expression changes associated with multiple clinical and neuropathological traits in 1,053 postmortem brain samples across 19 brain regions from 125 persons dying with varying severities of dementia and variable AD-neuropathology severities."]

In [6]:
gse.metadata['sample_id']

['GSM2233519',
 'GSM2233520',
 'GSM2233521',
 'GSM2233522',
 'GSM2233523',
 'GSM2233524',
 'GSM2233525',
 'GSM2233526',
 'GSM2233527',
 'GSM2233528',
 'GSM2233529',
 'GSM2233530',
 'GSM2233531',
 'GSM2233532',
 'GSM2233533',
 'GSM2233534',
 'GSM2233535',
 'GSM2233536',
 'GSM2233537',
 'GSM2233538',
 'GSM2233539',
 'GSM2233540',
 'GSM2233541',
 'GSM2233542',
 'GSM2233543',
 'GSM2233544',
 'GSM2233545',
 'GSM2233546',
 'GSM2233547',
 'GSM2233548',
 'GSM2233549',
 'GSM2233550',
 'GSM2233551',
 'GSM2233552',
 'GSM2233553',
 'GSM2233554',
 'GSM2233555',
 'GSM2233556',
 'GSM2233557',
 'GSM2233558',
 'GSM2233559',
 'GSM2233560',
 'GSM2233561',
 'GSM2233562',
 'GSM2233563',
 'GSM2233564',
 'GSM2233565',
 'GSM2233566',
 'GSM2233567',
 'GSM2233568',
 'GSM2233569',
 'GSM2233570',
 'GSM2233571',
 'GSM2233572',
 'GSM2233573',
 'GSM2233574',
 'GSM2233575',
 'GSM2233576',
 'GSM2233577',
 'GSM2233578',
 'GSM2233579',
 'GSM2233580',
 'GSM2233581',
 'GSM2233582',
 'GSM2233583',
 'GSM2233584',
 'GSM22335

In [7]:
for gsm_name, gsm in gse.gsms.items():
    print("Name: ", gsm_name)
    print("Metadata:",)
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print ("Table data:",)
    print (gsm.table.head())
    break

Name:  GSM2233519
Metadata:
 - title : Subject 495, region Amygdala
 - geo_accession : GSM2233519
 - status : Public on Aug 19 2016
 - submission_date : Jul 14 2016
 - last_update_date : Aug 20 2016
 - type : RNA
 - channel_count : 1
 - source_name_ch1 : Amygdala, 69 yr old male
 - organism_ch1 : Homo sapiens
 - taxid_ch1 : 9606
 - characteristics_ch1 : subject id: 495, age: 69, Sex: male, race: black, postmortem interval minutes: 255, ph: 6.3, clinical dementia rating: 0.5, braak neurofibrillary tangle score: 1, neuropathological category: Normal, average neuritic plaque density: 0, sum of cerad rating scores in multiple brain regions: 0, sum of neurofibrillary tangles density in multiple brain regions: 1, brain region: Amygdala, tissue: post-mortem brain
 - treatment_protocol_ch1 : N/A
 - growth_protocol_ch1 : N/A
 - molecule_ch1 : total RNA
 - extract_protocol_ch1 : Total RNA was isolated from 50 mg tissue aliquots from brain tissue samples, amplified to cRNA, and biotin-labeled for

In [8]:
pivoted_samples = gse.pivot_samples('VALUE')

In [9]:
pivoted_samples.head()

name,GSM2233519,GSM2233520,GSM2233521,GSM2233522,GSM2233523,GSM2233524,GSM2233525,GSM2233526,GSM2233527,GSM2233528,...,GSM2235513,GSM2235514,GSM2235515,GSM2235516,GSM2235517,GSM2235518,GSM2235519,GSM2235520,GSM2235521,GSM2235522
ID_REF,,,,,,,,,,,,,,,,,,,,,
1007_s_at,8.574104,8.991247,9.195859,9.362672,8.923102,9.382466,9.292576,9.122197,9.068594,9.066957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1053_at,4.770704,4.915865,5.436243,4.896158,4.842856,5.217553,4.761193,4.915572,4.420729,5.253055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117_at,2.722635,2.677848,2.619575,2.787066,2.750039,2.873187,2.342134,2.678053,2.444795,3.236246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121_at,2.708397,2.869098,2.843030,2.950779,3.205170,3.113110,2.913583,2.966424,2.798382,2.941285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1255_g_at,2.343187,3.312817,2.299937,2.920171,2.783208,2.800850,2.851128,2.812243,3.258507,2.553696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
pivoted_samples_average = pivoted_samples.median(axis=1)
print("Number of probes before filtering: ", len(pivoted_samples_average))

Number of probes before filtering:  54681


In [11]:
expression_threshold = pivoted_samples_average.quantile(0.25)

In [12]:
expressed_probes = pivoted_samples_average[pivoted_samples_average >= expression_threshold].index.tolist()
print("Number of probes above threshold: ", len(expressed_probes))

Number of probes above threshold:  41011


In [13]:
samples = gse.pivot_samples("VALUE").ix[expressed_probes]

/home/ranap/anaconda3/envs/BioSandbox/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [16]:
samples.head()

name,GSM2233519,GSM2233520,GSM2233521,GSM2233522,GSM2233523,GSM2233524,GSM2233525,GSM2233526,GSM2233527,GSM2233528,...,GSM2235513,GSM2235514,GSM2235515,GSM2235516,GSM2235517,GSM2235518,GSM2235519,GSM2235520,GSM2235521,GSM2235522
ID_REF,,,,,,,,,,,,,,,,,,,,,
1007_s_at,8.574104,8.991247,9.195859,9.362672,8.923102,9.382466,9.292576,9.122197,9.068594,9.066957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1053_at,4.770704,4.915865,5.436243,4.896158,4.842856,5.217553,4.761193,4.915572,4.420729,5.253055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117_at,2.722635,2.677848,2.619575,2.787066,2.750039,2.873187,2.342134,2.678053,2.444795,3.236246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121_at,2.708397,2.869098,2.843030,2.950779,3.205170,3.113110,2.913583,2.966424,2.798382,2.941285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1294_at,2.508205,2.133941,2.404052,2.541948,4.049023,2.443089,2.478089,2.504275,2.556177,2.320297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
print (gse.phenotype_data[["title", "source_name_ch1" , "characteristics_ch1.8.neuropathological category"]])

                                    title              source_name_ch1  \
GSM2233519   Subject 495, region Amygdala     Amygdala, 69 yr old male   
GSM2233520   Subject 551, region Amygdala   Amygdala, 85 yr old female   
GSM2233521   Subject 625, region Amygdala  Amygdala, 102 yr old female   
GSM2233522   Subject 776, region Amygdala     Amygdala, 95 yr old male   
GSM2233523   Subject 800, region Amygdala   Amygdala, 89 yr old female   
GSM2233524   Subject 532, region Amygdala   Amygdala, 92 yr old female   
GSM2233525   Subject 843, region Amygdala     Amygdala, 66 yr old male   
GSM2233526   Subject 923, region Amygdala     Amygdala, 69 yr old male   
GSM2233527   Subject 757, region Amygdala   Amygdala, 96 yr old female   
GSM2233528  Subject 1039, region Amygdala     Amygdala, 85 yr old male   
GSM2233529   Subject 985, region Amygdala     Amygdala, 78 yr old male   
GSM2233530  Subject 1009, region Amygdala     Amygdala, 76 yr old male   
GSM2233531  Subject 1024, region Amygd

In [18]:
experiments = {}
for i, (idx, row) in enumerate(gse.phenotype_data.iterrows()):
    tmp = {}
    tmp["Experiment"] = idx
    tmp["Type"] = row["characteristics_ch1.8.neuropathological category"] 
    experiments[i] = tmp
experiments = pd.DataFrame(experiments).T
print (experiments)

      Experiment         Type
0     GSM2233519       Normal
1     GSM2233520       Normal
2     GSM2233521       Normal
3     GSM2233522       Normal
4     GSM2233523       Normal
5     GSM2233524       Normal
6     GSM2233525       Normal
7     GSM2233526       Normal
8     GSM2233527       Normal
9     GSM2233528       Normal
10    GSM2233529       Normal
11    GSM2233530       Normal
12    GSM2233531       Normal
13    GSM2233532       Normal
14    GSM2233533       Normal
15    GSM2233534  definite AD
16    GSM2233535  definite AD
17    GSM2233536  Possible AD
18    GSM2233537  definite AD
19    GSM2233538  definite AD
20    GSM2233539  definite AD
21    GSM2233540  definite AD
22    GSM2233541  definite AD
23    GSM2233542  definite AD
24    GSM2233543  Possible AD
25    GSM2233544  definite AD
26    GSM2233545  Probable AD
27    GSM2233546  Probable AD
28    GSM2233547  definite AD
29    GSM2233548  Probable AD
...          ...          ...
1974  GSM2235493       Normal
1975  GSM2

In [19]:
for time, group in experiments.groupby("Type"):
    print(time)
    transfection_name = group[group.Type == "definite AD"]

Normal
Possible AD
Probable AD
definite AD


In [20]:
AD_sample = samples[transfection_name.Experiment]

In [21]:
AD_sample_annotated = AD_sample.reset_index().merge(gse.gpls['GPL570'].table[["ID", "ENTREZ_GENE_ID"]],
                                left_on='ID_REF', right_on="ID", how='inner')


In [22]:
AD_sample_annotated = AD_sample_annotated.dropna(subset=["ENTREZ_GENE_ID"])
# remove probes with more than one gene assigned
AD_sample_annotated = AD_sample_annotated[~AD_sample_annotated.ENTREZ_GENE_ID.str.contains("///")]
# for each gene average LFC over probes
AD_sample_annotated = AD_sample_annotated.groupby("ENTREZ_GENE_ID").median()

In [29]:
AD_sample_annotated.head()

,GSM2233534,GSM2233535,GSM2233537,GSM2233538,GSM2233539,GSM2233540,GSM2233541,GSM2233542,GSM2233544,GSM2233547,...,GSM2235488,GSM2235490,GSM2235495,GSM2235501,GSM2235503,GSM2235504,GSM2235512,GSM2235519,GSM2235521,GSM2235522
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
10,1.894549,1.844536,1.407503,1.851952,1.725851,1.752923,1.471343,1.395326,2.219697,1.463018,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
100,2.723472,2.660155,2.472315,2.179213,2.443815,2.617917,2.749893,2.385487,2.638806,2.430330,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1000,7.866425,7.966718,7.696251,7.666251,7.580768,8.220708,7.711140,8.136351,7.732809,8.177074,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
10000,3.490881,3.225545,3.508853,3.770085,3.404882,3.562278,3.537457,4.147778,3.415148,3.687742,...,3.626186,2.990150,3.002777,2.733827,4.026714,3.502457,3.413461,3.809056,3.650891,3.679745
100009676,1.728233,1.907187,1.772074,1.624281,1.816555,1.878175,1.637512,1.710690,1.577785,1.568170,...,2.598129,2.309836,2.343526,2.677604,2.299704,2.337758,2.815082,2.590416,2.605727,2.574401


In [24]:
AD_sample_annotated.tail()

,GSM2233534,GSM2233535,GSM2233537,GSM2233538,GSM2233539,GSM2233540,GSM2233541,GSM2233542,GSM2233544,GSM2233547,...,GSM2235488,GSM2235490,GSM2235495,GSM2235501,GSM2235503,GSM2235504,GSM2235512,GSM2235519,GSM2235521,GSM2235522
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
9991,3.373431,3.515525,3.530541,3.112780,3.719156,3.669874,3.416963,3.488313,3.396548,3.422685,...,4.098334,4.102293,4.339153,3.781243,4.487471,4.189845,4.008923,4.142997,4.526753,3.809142
9992,2.236065,1.828660,1.693131,2.564267,2.295832,1.713361,2.351161,1.862717,2.024900,1.999716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9993,4.369876,4.334250,4.433400,4.503829,4.340234,4.383432,4.170865,4.572300,4.080642,4.003531,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9994,5.940049,6.047123,6.041953,5.501499,6.262138,6.307817,5.719898,5.733741,6.062781,6.465161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,2.289445,2.576803,3.055829,2.829125,2.763506,3.016397,2.758843,3.679764,3.892756,3.055576,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
AD_sample_annotated.shape

(17284, 690)

In [31]:
AD_sample_annotated.fillna(1, inplace=True)

In [32]:
AD_sample_annotated.to_csv('AD_data_GEO.csv')